In [1]:
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

2022-04-17 17:53:23.871814: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import numpy as np # linear algebra
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
#from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.layers import BatchNormalization

In [3]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [5]:
X_train = X_train.reshape(len(X_train),32,32,3).astype('float32')
X_test = X_test.reshape(len(X_test),32,32,3).astype('float32')

In [6]:
X_train = X_train/255
X_test = X_test/255

In [7]:
n_classes=10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [8]:
def model():
    model = Sequential()

    model.add(Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 1)) # default stride is 2
    model.add(BatchNormalization())

    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 1)) # default stride is 2
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
             optimizer='nadam',
             metrics=['accuracy'])
    return model

In [9]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2022-04-17 17:56:37.585435: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-17 17:56:37.590402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-17 17:56:37.615125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-04-17 17:56:37.615172: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-17 17:56:37.896841: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-17 17:56:37.896956: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [10]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
BATCH_SIZE_PER_REPLICA = 500
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset =train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [12]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = model()
    gpu_model.fit(train_dataset, epochs=15)

2022-04-17 17:58:22.971603: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 10
        }
      }
    }
  }
}



Epoch 1/15
100/100 [==============================] - 13s 84ms/step - loss: 3.0437 - accuracy: 0.1900
Epoch 2/15
100/100 [==============================] - 9s 91ms/step - loss: 1.6232 - accuracy: 0.3883
Epoch 3/15
100/100 [==============================] - 9s 86ms/step - loss: 1.7434 - accuracy: 0.3865
Epoch 4/15
100/100 [==============================] - 9s 91ms/step - loss: 0.9999 - accuracy: 0.6489
Epoch 5/15
100/100 [==============================] - 9s 86ms/step - loss: 0.7694 - accuracy: 0.7379
Epoch 6/15
100/100 [==============================] - 9s 92ms/step - loss: 0.6155 - accuracy: 0.7927
Epoch 7/15
100/100 [==============================] - 9s 86ms/step - loss: 0.4961 - accuracy: 0.8340
Epoch 8/15
100/100 [==============================] - 9s 93ms/step - loss: 0.4112 - accuracy: 0.8653
Epoch 9/15
100/100 [==============================] - 8s 85ms/step - loss: 0.3450 - accuracy: 0.8877
Epoch 10/15
100/100 [==============================] - 9s 92ms/step - loss: 0.2718 - accur

In [50]:
tf.config.experimental.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [59]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import argparse

# the default is using two gpus 0 and 1

batch_size = 128
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(),])
)

ModuleNotFoundError: No module named 'torch'